In [52]:
import requests
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re
from datetime import datetime
import time

In [ ]:
import configparser

config = configparser.ConfigParser()
config.read('./config.ini')

db_host = config['database']['host']
db_user = config['database']['user']
db_password = config['database']['password']
db_database = config['database']['database']
db_charset = config['database']['charset']


openai_key = config['openai']['OPENAI_KEY']

In [104]:
import sys

import csv
import pymysql


connection = pymysql.connect(
    host=db_host,
    user=db_user,
    password=db_password,
    database=db_database,
    charset=db_charset,
    cursorclass=pymysql.cursors.DictCursor
)
# db에 연결
cursor = connection.cursor()

current_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

delete_query = "DELETE FROM event_summary WHERE card_company_id = 3"

cursor.execute(delete_query)
connection.commit()

In [105]:
import openai
import json
openai.api_key=openai_key
prompt = "입력된 데이터는 카드사의 이벤트의 혜택 정보를 크롤링한 텍스트입니다. 데이터를 [] 사이에 주어진 key를 가지는 JSON 형식의 list로 요약하여 반환합니다. [{benefit_field, content, startdate, enddate}]\\\\benefit_field는 혜택의 분야(브랜드 포함), content는 대상카드와 혜택 할인율 정보를 핵심만 요약하여 나타냅니다. startdate는 해당 이벤트의 시작날짜를, enddate는 해당 이벤트의 끝나는 날짜를 %Y-%m-%d형식으로 반환하며, 해당 정보가 없는 경우 null로 반환합니다. 현재는 2024년입니다."
def Summary(contents):
    response = openai.ChatCompletion.create(
        model = 'gpt-3.5-turbo-0125',
        messages=[
            {"role": "system", "content":  prompt},
            {"role": "user", "content": contents},
        ],
    
        temperature=0
    )

    return response["choices"][0]["message"]["content"]

In [106]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
chrome_options = Options()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument("--disable-dev-shm-usage")

# 서버
# service = Service(executable_path='/usr/bin/chromedriver')
# driver = webdriver.Chrome(service=service,options=chrome_options)
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
driver.implicitly_wait(20)




In [107]:
driver.get('https://www.samsungcard.com/personal/event/ing/UHPPBE1401M0.jsp')
time.sleep(3)
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

In [108]:
links = soup.find('div', {'class':'list_evt'}).findAll('li')

In [109]:
for link in links:
    event_link='https://www.samsungcard.com/personal/event/ing/UHPPBE1403M0.jsp?cms_id='+link.get('id')
    print(event_link)
    driver.get(event_link)
    time.sleep(3)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    summary_contents=Summary(soup.find('dl',{'class':'form_event_new'}).text)
    print(summary_contents)
    data = json.loads(summary_contents)
    for item in data:
        benefit_field = item['benefit_field']
        content = item['content']
        startdate = item['startdate']
        enddate = item['enddate']
        cursor.execute("""
            INSERT INTO `event_summary`(event_field,event_contents,expire_date,start_date,card_company_id)
            VALUES  ('%s', '%s', '%s', '%s', '%d') 
            """%(benefit_field,content,startdate,enddate,3))
        connection.commit()

https://www.samsungcard.com/personal/event/ing/UHPPBE1403M0.jsp?cms_id=3648984
[{"benefit_field": "Mastercard 삼성개인신용카드", "content": "항공권 70,000원 즉시할인(25만원 이상 결제 시), 이용금액대별 상품권 증정, 플라잉 블루 웰컴 보너스 10,000 마일리지", "startdate": "2024-05-14", "enddate": "2025-03-31"}]
https://www.samsungcard.com/personal/event/ing/UHPPBE1403M0.jsp?cms_id=3649434
[{"benefit_field": "신라인터넷면세점", "content": "삼성개인신용카드 결제 시 15천원/25천원/50천원/200천원 할인", "startdate": "2024-05-13", "enddate": "2024-05-21"}]
https://www.samsungcard.com/personal/event/ing/UHPPBE1403M0.jsp?cms_id=3636970
[
    {
        "benefit_field": "트레이더스 제휴",
        "content": "삼성카드 5종(가족카드 포함) 현장할인",
        "startdate": "2024-05-13",
        "enddate": "2024-05-19"
    }
]
https://www.samsungcard.com/personal/event/ing/UHPPBE1403M0.jsp?cms_id=3648059
[{"benefit_field": "삼성화재 다이렉트 영업용 개인 화물차보험", "content": "스타벅스 아메리카노 2잔 즉시 제공", "startdate": "2024-05-10", "enddate": "2024-05-31"}]
https://www.samsungcard.com/personal/event/ing/UHPPBE1403M0.jsp?cms_id

In [34]:
print(links)

[<li id="3648984"> <a class="m_link" href="javascript:$.noop();" onclick="javascript:UHPPBE1401M0_EVENT.GoDtlBrws('3648984', '00', 'N');"> <span class="like" href="#"> <span class="hide">좋아요 갯수</span>4</span> <div class="img"> <img alt="" class="p_display" src="//static11.samsungcard.com/wcms/event/general_v2/__icsFiles/afieldfile/2024/05/13/P_thumb_286x275.png"/><!-- [pc] img size : 284*275 --> <img alt="" class="m_display" src="//static11.samsungcard.com/wcms/event/general_v2/__icsFiles/afieldfile/2024/05/13/M_thumb_320x320.png"/><!-- [mobile] img size : 316*316 --> </div> <div class="cont"> <p class="tit">해외 여행은 에어프랑스로 항공권 7만원 할인</p> <span class="date">2024.05.14~2024.06.13</span> </div> </a> </li>, <li id="3649434"> <a class="m_link" href="javascript:$.noop();" onclick="javascript:UHPPBE1401M0_EVENT.GoDtlBrws('3649434', '00', 'N');"> <span class="like" href="#"> <span class="hide">좋아요 갯수</span>13</span> <div class="img"> <img alt="" class="p_display" src="//static11.samsungcard.com

In [93]:
url="https://www.samsungcard.com/personal/event/ing/UHPPBE1403M0.jsp?cms_id=3647525"
# html=urlopen(url)
# bs=BeautifulSoup(html,'html.parser')

In [94]:
driver.get(url)
time.sleep(3)
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

In [95]:
print(soup.find('dl',{'class':'form_event_new'}).text)

 대상카드 삼성개인신용카드(가족카드 포함) 참여방법 스마일페이에 등록된 대상카드로 아래 행사 가맹점에서 결제 




 지마켓/옥션
 5/7(화) ~ 5/20(월) 
100,000원 이상 결제 시 7% 즉시할인 (1일 1회, 할인한도 70,000원) 

카드할인 태그 상품에 한해 혜택 제공   
  

   
 지마켓
                    바로가기 옥션 바로가기 지마켓 바로가기 옥션 바로가기   
  


In [102]:
test=Summary(soup.find('dl',{'class':'form_event_new'}).text)

In [103]:
print(test)

[{"benefit_field": "지마켓/옥션", "content": "삼성개인신용카드(가족카드 포함) - 7% 즉시할인 (1일 1회, 할인한도 70,000원)", "startdate": "2019-05-07", "enddate": "2019-05-20"}]


In [ ]:
cursor.close()
connection.close()